In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
video_path = 'hw2_1/news.mpg'
output_folder = 'output/'

if os.path.isdir(output_folder):
    print("Delete old result folder: {}".format(output_folder))
    os.system("rm -rf {}".format(output_folder))
os.system("mkdir {}".format(output_folder))
print("create folder: {}".format(output_folder))

vc = cv2.VideoCapture(video_path)
fps = vc.get(cv2.CAP_PROP_FPS)
frame_count = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_count)
video = []


create folder: output/
1380


In [8]:
#vc.set(1, 1)
ret, frame = vc.read()
height, width, layers = frame.shape
size = (width, height)
if frame is not None:
    cv2.imshow('window',frame)
else:
    print('error')